---
title: RToken Backtester 
description: Backtest RToken yields with Defillama historical data
show-code: False
params:
    is_custom_weights:
        label: Use custom basket weights (default is income-weighted)
        input: checkbox
        value: False    
    basket_weights:
        input: text
        label: Basket weights (comma-separated as %)
    basket_tokens:
        input: select
        label: Select collateral assets
        multi: True
        choices: ['MIM-3CRV (Convex)', 'FRAX-USDC (Convex)', 'ALUSD-3CRV (Convex)', 'OHM-FRAXBP (Curve)', 'DAI (Compound)', 'USDC (Compound)', 'USDC (Aave)', 'USDT (Aave)', 'BUSD (Aave)', 'USDC-USDT (Uniswap V3)', 'BUSD-USDC (Uniswap V3)', 'USDC-USDT (Uniswap V2)', 'OHM-DAI (Balancer)', 'OHM-WETH (Balancer)', 'USDC (Goldfinch)', 'USDC (Stargate)', 'STETH']
    reference_token:
        input: select
        label: Select reference asset
        choices: ['MIM-3CRV (Convex)', 'FRAX-USDC (Convex)', 'ALUSD-3CRV (Convex)', 'OHM-FRAXBP (Curve)', 'DAI (Compound)', 'USDC (Compound)', 'USDC (Aave)', 'USDT (Aave)', 'BUSD (Aave)', 'USDC-USDT (Uniswap V3)', 'BUSD-USDC (Uniswap V3)', 'USDC-USDT (Uniswap V2)', 'OHM-DAI (Balancer)', 'OHM-WETH (Balancer)', 'USDC (Goldfinch)', 'USDC (Stargate)', 'STETH']
---

In [ ]:
import pandas as pd
import numpy as np
import requests as req
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pool_dict = {
    "MIM-3CRV (Convex)": "8a20c472-142c-4442-b724-40f2183c073e",
    "FRAX-USDC (Convex)": "bd072651-d99c-4154-aeae-51f12109c054",
    "ALUSD-3CRV (Convex)": "f11aa021-e4c1-49f9-8004-9cf5625dedae",
    "OHM-FRAXBP (Curve)": "652ec67f-20b1-4138-8fc3-d78458f47e1a",
    "DAI (Compound)": "cc110152-36c2-4e10-9c12-c5b4eb662143",
    "USDC (Compound)": "7da72d09-56ca-4ec5-a45f-59114353e487",
    "USDC (Aave)": "a349fea4-d780-4e16-973e-70ca9b606db2",
    "USDT (Aave)": "60d657c9-5f63-4771-a85b-2cf8d507ec00",
    "BUSD (Aave)": "fd700196-ace0-4adc-8c91-6da04acb4ec3",
    "USDC-USDT (Uniswap V3)": "e737d721-f45c-40f0-9793-9f56261862b9",
    "BUSD-USDC (Uniswap V3)": "4bb8783d-9919-4a8e-980e-546401a67f63",
    "USDC-USDT (Uniswap V2)": "e2c09021-0aa8-42b6-a596-5422c2e35399",
    "OHM-DAI (Balancer)": "645dffc4-fe90-4c8b-9c17-fd8f2143a5fe",
    "OHM-WETH (Balancer)": "ad68c6ce-699a-4c00-986f-5f92e38c1ac1",
    "USDC (Goldfinch)": "c89af47f-032a-4c59-bdc9-17a7f1514a3b",
    "USDC (Stargate)": "c24d9138-51aa-48f4-a445-7f33bf7bf5fb",
    "STETH": "747c1d2a-c668-4682-b9f9-296708a3dd90"
}


In [ ]:
# Raw pool data in dictionaries 

pool_dict_raw = dict(pd.DataFrame(req.get('https://yields.llama.fi/pools').json())['data'])

# Dataframe of raw data
df_raw = pd.DataFrame.from_dict(pool_dict_raw, orient='index')
df_raw = df_raw.sort_values(by ='tvlUsd', ascending = False)
eth_pools = df_raw[df_raw.chain == 'Ethereum']

In [ ]:
# Set Parameters for Basket

min_apy = 0
is_stablecoin = True
min_tvl = 80000000
top_x = 15
sort_by = 'apy' # or tvlUsd or apy

In [ ]:
basket_tokens = []
reference_token = "USDC (Compound)"
basket_weights = ""
is_custom_weights = False

In [ ]:
using_custom_weights = is_custom_weights and len(basket_tokens) == len(basket_weights.split(','))

benchmark = pool_dict[reference_token]
bespoke = [pool_dict[token] for token in basket_tokens]
weights = [float(weight) if using_custom_weights else 0 for weight in basket_weights.split(',')]
normalized_weights = [weight/sum(weights) for weight in weights if using_custom_weights]

pool_symbols = eth_pools[eth_pools.pool.isin(bespoke)][['project','symbol','pool', 'tvlUsd','apy']]
benchmark_symbols = eth_pools[eth_pools.pool == benchmark][['project','symbol','pool', 'tvlUsd','apy']]

In [ ]:
# Creates a table with pool symbol and pool ids to be later used for joining 

def symbol_ops(df):
    # calc for income
    df['income'] = df.tvlUsd * df.apy

    # calc for income weighted basket based on current TVL and APY 
    df['income_basket_weight'] = round(df.income / sum(df.income),2)
        
    # concat the pool symbol to show pool and protocol
    df['pool (protocol)'] = df.symbol + " (" + df.project + ")"

    #drop columns that we no longer need
    df= df.drop(['tvlUsd','apy','income'], axis=1)
    
    return df
    

# Establishes a list of dataframes of historical data for each of the pools in the desired basket
# Input is the dataframe returned by symbol_ops() 

def create_historical_dfs(df):
    basket_pool_ids = list(df.pool)
    
    pool_dfs = []
    for i in basket_pool_ids:
        data = pd.DataFrame()
        data = pd.DataFrame(req.get('https://yields.llama.fi/chart/'+ str(i)).json()['data'])
        data = data[['timestamp','tvlUsd','apy']]
        data['timestamp'] = [i[0:10] for i in list(data.timestamp)]
        data['timestamp'] = pd.to_datetime(data.timestamp)
        data['pool_id'] = str(i)

        # joins pool symbol onto dataframe using pool id
        data = pd.merge(data, df, how='left', 
             left_on= 'pool_id', right_on='pool' )[['timestamp','tvlUsd','apy','income_basket_weight','pool (protocol)']]

        # adds column for portfolio yield contribution (apy * income_basket_weight)
        data['portfolio_yield_contribution'] = data.iloc[:,2] * data.iloc[:,3]

        # renames columns to reflect pool symbol and drops pool symbol column
        pool_protocol_string = data['pool (protocol)'][0]
        data.columns = ['timestamp',  
                          ('tvlUsd | ' + pool_protocol_string), 
                           ('apy | ' + pool_protocol_string),
                        'income_basket_weight | ' + pool_protocol_string,
                        'pool (protocol)',
                       'portfolio_yield_contribution | ' + pool_protocol_string]
        data = data.drop('pool (protocol)', axis=1)

        # creates and calculates column for income (apy column has more decimal points than shown, results in slight math discrepency on income)
        #income_column_string = 'income | ' + pool_protocol_string
        #data[income_column_string] = data.iloc[:, 1] * data.iloc[:, 2] / 100

        pool_dfs.append(data)
        
    return pool_dfs


#Checks to see which pool within the basket has the least amount of historical data
# Input is the list of dfs returned by create_historical_dfs()
def count_days_by_pool(pool_dfs_list):
    for i in pool_dfs_list:
        print( (list(i)[1][8:], ' : ', len(i)))


        
# Creates master dataframe with historical data froma all pools, using the pool that has the least hisory as base, 
# and caluculates the total_portfolio_yield for each day

# Input is the list of dfs returned by create_historical_dfs()

def calc_master_df(pool_dfs_list):
    from functools import reduce
    df_master = reduce(lambda x, y: pd.merge(x, y, on = 'timestamp', how ='left'), sorted(pool_dfs_list, key=len))
    
    yield_contribution_cols = [col for col in df_master.columns if 'portfolio_yield_contribution |' in col]

    # caluculates the total_portfolio_yield for each day
    df_master['total_portfolio_yield'] = df_master[yield_contribution_cols].sum(axis=1)
    
    return df_master

In [ ]:
pool_symbols = symbol_ops(pool_symbols)
if(using_custom_weights):
    pool_symbols['income_basket_weight'] = normalized_weights
    
benchmark_symbols = symbol_ops(benchmark_symbols)


## RToken Basket Composition

In [ ]:
pool_symbols

## Benchmark

In [ ]:
benchmark_symbols

In [ ]:
pool_dfs_basket = create_historical_dfs(pool_symbols)
pool_dfs_benchmark = create_historical_dfs(benchmark_symbols)

In [ ]:
df = calc_master_df(pool_dfs_basket)

df_benchmark = calc_master_df(pool_dfs_benchmark)

# makes basket and benchmark dataframes the same length
num_days_of_data = min(len(df),len(df_benchmark))

df = df.tail(num_days_of_data)
df_benchmark = df_benchmark.tail(num_days_of_data)

## Backtest Results

In [ ]:
_ = sns.lineplot(x = "timestamp", y = "total_portfolio_yield",
             data = df)
_ = sns.lineplot(x = "timestamp", y = "total_portfolio_yield",
                data = df_benchmark, color="orange")
_ = plt.title('Benchmark Yield Over Time')
_ = plt.ylabel('APY')
_ = plt.xlabel('Date')
_ = plt.legend(['RToken Basket Yield','Benchmark'])
_ = plt.xticks(rotation = 45)
